# Imports

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!mv yolov5/* ./

fatal: destination path 'yolov5' already exists and is not an empty directory.
mv: cannot stat 'yolov5/*': No such file or directory


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mkdir fulltrain

mkdir: cannot create directory ‘fulltrain’: File exists


In [ ]:
mkdir fulltrain/dataset

mkdir: cannot create directory ‘fulltrain/dataset’: File exists


In [ ]:
!unzip /content/drive/MyDrive/SDAIA/images2.zip -d fulltrain/dataset/images2

In [ ]:
import torch
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2

## getting image shapes

In [ ]:
heights = []
widths = []

for i in os.listdir('./dataset/images/'):
    sh  = cv2.imread(f"./dataset/images/{i}").shape
    if sh[0] not in heights:
        heights.append(sh[0])
    if sh[1] not in widths:
        widths.append(sh[1])

print(len(heights),heights)
print(len(widths),widths)

* all images have the same shape

## resizing images

In [ ]:
df = pd.read_csv('./fulltrain/dataset/train.csv')
df.head()

In [ ]:
img = cv2.imread(f'./dataset/images/{df.iloc[0]["image_path"]}')
cv2.rectangle(img,(int(df.iloc[0]['xmin']*2),int(df.iloc[0]['ymin']*2)),(int(df.iloc[0]['xmax']*2),int(df.iloc[0]['ymax']*2)), (0,0,0), 2)
cv2.rectangle(img,(int(df.iloc[1]['xmin']*2),int(df.iloc[1]['ymin']*2)),(int(df.iloc[1]['xmax']*2),int(df.iloc[1]['ymax']*2)), (0,0,0), 2)
cv2.rectangle(img,(int(df.iloc[2]['xmin']*2),int(df.iloc[2]['ymin']*2)),(int(df.iloc[2]['xmax']*2),int(df.iloc[2]['ymax']*2)), (0,0,0), 2)
plt.imshow(img)
print(img.shape)
ky = 640/1080
kx = 640/1920
img2 = cv2.resize(img, (640,640))

cv2.rectangle(img2,(int(df.iloc[0]['xmin']*2*kx),int(df.iloc[0]['ymin']*2*ky)),(int(df.iloc[0]['xmax']*2*kx),int(df.iloc[0]['ymax']*2*ky)), (0,0,0), 2)
cv2.rectangle(img2,(int(df.iloc[1]['xmin']*2*kx),int(df.iloc[1]['ymin']*2*ky)),(int(df.iloc[1]['xmax']*2*kx),int(df.iloc[1]['ymax']*2*ky)), (0,0,0), 2)
cv2.rectangle(img2,(int(df.iloc[2]['xmin']*2*kx),int(df.iloc[2]['ymin']*2*ky)),(int(df.iloc[2]['xmax']*2*kx),int(df.iloc[2]['ymax']*2*ky)), (0,0,0), 2)
plt.imshow(img2)

In [ ]:
def resize_image(path,xmin,ymin,xmax,ymax):
    img = cv2.imread(f'./fulltrain/dataset/images/{path}')
    ky = 640/1080
    kx = 640/1920
    img2 = cv2.resize(img, (640,640))
    cv2.imwrite(f"./fulltrain/dataset/images2/{path}",img2)
    return xmin*kx*2,ymin*ky*2,xmax*kx*2,ymax*ky*2


In [ ]:
imgs = []
xmins = []
ymins = []
xmaxs = []
ymaxs = []
classes = []
test_imgs = []
images = os.listdir('./fulltrain/dataset/images/')
for i in range(len(images)):
    im = df[df['image_path'] == images[i]]
    if len(im)>0:
        for j in range(len(im)):
            xmin,ymin,xmax,ymax = resize_image(im.iloc[j]['image_path'],im.iloc[j]['xmin'],im.iloc[j]['ymin'],im.iloc[j]['xmax'],im.iloc[j]['ymax'])
            xmins.append(xmin)
            xmaxs.append(xmax)
            ymins.append(ymin)
            ymaxs.append(ymax)
            imgs.append(im.iloc[j]['image_path'])
            classes.append(im.iloc[j]['class'])
    else:
        test_imgs.append(images[i])

In [ ]:
test_df = pd.read_csv('./fulltrain/dataset/test.csv')['image_path']
set(test_df).difference(set(test_imgs))

In [ ]:
df2 = pd.DataFrame({'class':classes,'image_path':imgs,'xmin':xmins,'ymin':ymins,'xmax':xmaxs,'ymax':ymaxs})

In [ ]:
df2['class'].value_counts()

In [ ]:
df2.to_csv('./fulltrain/dataset/train2.csv')

## augmentation

In [ ]:
del imgs,xmins,ymins,xmaxs,ymaxs,classes,test_imgs,df2

## pascal to yolo

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SDAIA/train2.csv')
img_width = 640
img_height =  640
df['x_center'] = (df['xmin'] + df['xmax']) / (2*img_width)
df['y_center'] = (df['ymin'] + df['ymax']) / (2*img_height)
df['width'] = (df['xmax'] - df['xmin']) / (2*img_width)
df['height'] = (df['ymax'] - df['ymin']) / (2*img_height)

In [ ]:
df =df[['image_path','x_center','y_center','width','height','class']]

In [ ]:
df.head()

,image_path,x_center,y_center,width,height,class
0,dbc81b50db7a67e60f7ed517ff5ac88d.jpg,0.078125,0.403841,0.041319,0.089986,7.0
1,dbc81b50db7a67e60f7ed517ff5ac88d.jpg,0.029514,0.411523,0.025347,0.073525,7.0
2,499f60c6f967c7a8961a2ec34cb03f72.jpg,0.125347,0.539918,0.028819,0.063649,3.0
3,499f60c6f967c7a8961a2ec34cb03f72.jpg,0.024653,0.603567,0.028125,0.070233,3.0
4,d8f7ca47e6629ab30c6dffd908187f2d.jpg,0.563194,0.121811,0.058333,0.124005,7.0


In [ ]:
df.describe()

,x_center,y_center,width,height,class
count,19950.000000,19950.000000,19950.000000,19950.000000,19950.000000
mean,0.324733,0.565945,0.073054,0.123663,4.055388
std,0.172769,0.187616,0.062048,0.086871,2.502491
min,0.007639,0.016461,0.000694,0.001097,0.000000
25%,0.172917,0.441152,0.031597,0.068038,3.000000
50%,0.322222,0.542112,0.050000,0.096571,3.000000
75%,0.473611,0.678189,0.090972,0.150343,4.000000
max,0.658333,1.150069,0.609722,0.813169,10.000000


## training

### train/val split

In [ ]:
# index = list(set(df.image_path))
# print(len(index)) # stratify
# fold=0
# val_index = index[len(index)*fold//5:len(index)*(fold+1)//35]
# print(len(val_index))
# val_index

In [ ]:
# !rm -r ./convertor

In [ ]:
from tqdm.auto import tqdm
import shutil as sh

index = list(set(df.image_path))
len(index)

source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//35]
        for name,mini in tqdm(df.groupby('image_path')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name.split('.')[0]+".txt", 'w+') as f:
                row = mini[['class','x_center','y_center','width','height']].astype(float).values
                # row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("./fulltrain/dataset/images2/images2/{}".format(name),'convertor/fold{}/images/{}/{}'.format(fold,path2save,name))

  0%|          | 0/7874 [00:00<?, ?it/s]

In [ ]:
%%writefile ./convertor/wheat0.yaml

# COCO 2017 dataset http://cocodataset.org - first 128 training images
# Download command:  python -c "from yolov5.utils.google_utils import gdrive_download; gdrive_download('1n_oKgR81BJtqk75b00eAjdv03qVCQn2f','coco128.zip')"
# Train command: python train.py --data ./data/coco128.yaml
# Dataset should be placed next to yolov5 folder:
#   /parent_folder
#     /coco128
#     /yolov5


# train and val datasets (image directory or *.txt file with image paths)
train: ./convertor/fold0/images/train2017/
val: ./convertor/fold0/images/val2017/

# number of classes
nc: 11

# class names
# names: ['graffiti', 'faded_signage', 'potholes', 'garbage', 'construction_road', 'broken_signage', 'bad_streetlight', 'bad_billboard', 'sand_on_road', 'clutter_sidewalk', 'unkept_facade']
names: ['0.0','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0']

Overwriting ./convertor/wheat0.yaml


In [ ]:
%%writefile ./convertor/yolov5x.yaml
nc: 11
depth_multiple: 0.33
width_multiple: 0.50
anchors:
  - [10,13, 16,30, 33,23] 
  - [30,61, 62,45, 59,119]
  - [116,90, 156,198, 373,326]
backbone:
  [[-1, 1, Focus, [64, 3]],
   [-1, 1, Conv, [128, 3, 2]],
   [-1, 3, Bottleneck, [128]],
   [-1, 1, Conv, [256, 3, 2]],
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]], 
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 6, BottleneckCSP, [1024]],
  ]
head:
  [[-1, 3, BottleneckCSP, [1024, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],
   [-2, 1, nn.Upsample, [None, 2, "nearest"]],
   [[-1, 6], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 3, BottleneckCSP, [512, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],
   [-2, 1, nn.Upsample, [None, 2, "nearest"]],
   [[-1, 4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 3, BottleneckCSP, [256, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],
[[], 1, Detect, [nc, anchors]],
  ]

Overwriting ./convertor/yolov5x.yaml


In [ ]:
!python train.py --img 640 --batch 16 --epochs 10 --data convertor/wheat0.yaml --name yolov5x_fold0 --weights '/content/yolov5s.pt'

In [ ]:
# rm -r './content/fulltrain/'

### save weights

In [ ]:
import shutil
shutil.copy("./runs/train/yolov5x_fold0/weights/best.pt", "/content/drive/MyDrive/SDAIA/weights/best2.pt")

'/content/drive/MyDrive/SDAIA/weights/best2.pt'

# detection

In [ ]:
# !unzip /content/drive/MyDrive/SDAIA/test_images.zip -d fulltrain/dataset/test_images

In [ ]:
dff = pd.DataFrame({'image':[],'xmin':[],'ymin':[],'xmax':[],'ymax':[],'name':[]})

In [ ]:
model = torch.hub.load('/content/', 'custom', path='/content/drive/MyDrive/SDAIA/weights/best2.pt', source='local')  # local repo

YOLOv5 🚀 2023-1-17 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
#Model
# Images
images = os.listdir('./fulltrain/dataset/test_images/test_images/')
for i in images:
  img = cv2.imread(f'./fulltrain/dataset/test_images/test_images/{i}')
  # Inference
  results = model(img, size=640)  # includes NMS
  # infs.append()  # img1 predictions (tensor)
  try:
    results = results.pandas().xyxy[0].iloc[0]
    df = pd.DataFrame({'image':[i],'xmin':[results['xmin']],'ymin':[results['ymin']],'xmax':[results['xmax']],'ymax':[results['ymax']],'name':[results['name']]})
    dff = pd.concat([dff,df],axis=0)
  except Exception as e:
    df = pd.DataFrame({'image':[i],'xmin':[0],'ymin':[0],'xmax':[0],'ymax':[0],'name':[None]})
    dff = pd.concat([dff,df],axis=0)
  

In [ ]:
dff.head()

,image,xmin,ymin,xmax,ymax,name
0,8b2c77725e46f1fc311225138275db82.jpg,5.516815,156.495422,142.050217,442.031555,4.0
0,e846938b21e4aba9419da248044cb4d3.jpg,0.000000,0.000000,0.000000,0.000000,None
0,a4022e05023a8c4469cae024e4a77957.jpg,0.000000,218.669189,149.684174,450.843872,4.0
0,dbfc319364fdc777df4e97e1a6e0d4a6.jpg,0.000000,0.000000,0.000000,0.000000,None
0,758e9f1cc7aa08b5027e5614029da39f.jpg,10.766762,231.999680,138.862183,390.966309,4.0


In [ ]:
dff['name'].value_counts()

4.0    141
3.0     13
Name: name, dtype: int64

In [ ]:
dff['name'].value_counts()

4.0    141
3.0     13
Name: name, dtype: int64

In [ ]:
clas_name = {'0.0':'GRAFFITI',
        '1.0':'FADED_SIGNAGE',
        '2.0':'POTHOLES',
        '3.0':'GARBAGE',
        '4.0':'CONSTRUCTION_ROAD',
        '5.0':'BROKEN_SIGNAGE',
        '6.0':'BAD_STREETLIGHT',
        '7.0':'BAD_BILLBOARD',
        '8.0':'SAND_ON_ROAD',
        '9.0':'CLUTTER_SIDEWALK',
        '10.0':'UNKEPT_FACADE',
        'None':'GARBAGE'}

# inv_map = {v: k for k, v in my_map.items()}
name_clas = {'GRAFFITI':'0.0',
        'FADED_SIGNAGE':'1.0',
        'POTHOLES':'2.0',
        'GARBAGE':'3.0',
        'CONSTRUCTION_ROAD':'4.0',
        'BROKEN_SIGNAGE':'5.0',
        'BAD_STREETLIGHT':'6.0',
        'BAD_BILLBOARD':'7.0',
        'SAND_ON_ROAD':'8.0',
        'CLUTTER_SIDEWALK':'9.0',
        'UNKEPT_FACADE':'10.0',}

In [ ]:
dff['name'] = dff['name'].apply(lambda x: clas_name[str(x)])

In [ ]:
dff['class'] = dff['name'].apply(lambda x: name_clas[str(x)])

In [ ]:
dff.head()

,image,xmin,ymin,xmax,ymax,name,class
0,8b2c77725e46f1fc311225138275db82.jpg,5.516815,156.495422,142.050217,442.031555,CONSTRUCTION_ROAD,4.0
0,e846938b21e4aba9419da248044cb4d3.jpg,0.000000,0.000000,0.000000,0.000000,GARBAGE,3.0
0,a4022e05023a8c4469cae024e4a77957.jpg,0.000000,218.669189,149.684174,450.843872,CONSTRUCTION_ROAD,4.0
0,dbfc319364fdc777df4e97e1a6e0d4a6.jpg,0.000000,0.000000,0.000000,0.000000,GARBAGE,3.0
0,758e9f1cc7aa08b5027e5614029da39f.jpg,10.766762,231.999680,138.862183,390.966309,CONSTRUCTION_ROAD,4.0


In [ ]:
dff = dff[['class', 'image', 'name', 'xmax', 'xmin','ymax','ymin']]

In [ ]:
dff.rename(columns = {'image':'image_path'}, inplace = True)

In [ ]:
dff.head()

,class,image_path,name,xmax,xmin,ymax,ymin
0,4.0,8b2c77725e46f1fc311225138275db82.jpg,CONSTRUCTION_ROAD,142.050217,5.516815,442.031555,156.495422
0,3.0,e846938b21e4aba9419da248044cb4d3.jpg,GARBAGE,0.000000,0.000000,0.000000,0.000000
0,4.0,a4022e05023a8c4469cae024e4a77957.jpg,CONSTRUCTION_ROAD,149.684174,0.000000,450.843872,218.669189
0,3.0,dbfc319364fdc777df4e97e1a6e0d4a6.jpg,GARBAGE,0.000000,0.000000,0.000000,0.000000
0,4.0,758e9f1cc7aa08b5027e5614029da39f.jpg,CONSTRUCTION_ROAD,138.862183,10.766762,390.966309,231.999680


# resize to 960

In [ ]:
def resize_image(path,xmin,ymin,xmax,ymax):
    img = cv2.imread(f'./dataset/images/{path}')
    ky = 640/1080
    kx = 640/1920
    img2 = cv2.resize(img, (640,640))
    cv2.imwrite(f"./dataset/images2/{path}",img2)
    return xmin*kx*2,ymin*ky*2,xmax*kx*2,ymax*ky*2


In [ ]:
kx = 960/640
ky = 960/640
print(kx,ky)

1.5 1.5


In [ ]:
dff['xmax'] = dff['xmax']*kx
dff['xmin'] = dff['xmin']*kx
dff['ymax'] = dff['ymax']*ky
dff['ymin'] = dff['ymin']*ky

In [ ]:
dff.head()

,class,image_path,name,xmax,xmin,ymax,ymin
0,4.0,8b2c77725e46f1fc311225138275db82.jpg,CONSTRUCTION_ROAD,213.075325,8.275223,663.047333,234.743134
0,3.0,e846938b21e4aba9419da248044cb4d3.jpg,GARBAGE,0.000000,0.000000,0.000000,0.000000
0,4.0,a4022e05023a8c4469cae024e4a77957.jpg,CONSTRUCTION_ROAD,224.526260,0.000000,676.265808,328.003784
0,3.0,dbfc319364fdc777df4e97e1a6e0d4a6.jpg,GARBAGE,0.000000,0.000000,0.000000,0.000000
0,4.0,758e9f1cc7aa08b5027e5614029da39f.jpg,CONSTRUCTION_ROAD,208.293274,16.150143,586.449463,347.999519


In [ ]:
dff.to_csv('./drive/MyDrive/SDAIA/inference2.csv',index=False)